# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pickle

import time
from contextlib import contextmanager

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacob.punter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jacob.punter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacob.punter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
@contextmanager
def timer(process):
    t0 = time.time()
    yield
    if int(time.time() - t0) < 60:
        print('{} executed in {} seconds\n'.format(
            process, int(time.time() - t0)))
    else:
        print('{} executed in {} minutes\n'.format(
            process, int(time.time() - t0) / 60))

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
connection = engine.connect()
df = Table('labelled_messages', MetaData(), autoload=True, autoload_with=engine)
df = pd.read_sql_table('labelled_messages', connection)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
id                        26216 non-null int64
message                   26216 non-null object
original                  10170 non-null object
genre                     26216 non-null object
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null i

In [5]:
X = df['message']
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):  
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_tokens.append(lemmatizer.lemmatize(token).lower().strip())
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words('english')]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
with timer("Fit"):
    untuned_model = pipeline.fit(X_train, y_train)

Fit executed in 1.0666666666666667 minutes



In [11]:
pickle.dump(untuned_model, open("untuned_model.pkl", "wb"))

Training the pipeline took 1 minute, so if we have 30 parameter combinations we can expect it to take around half an hour

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_columns = ['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']

In [13]:
def class_report_values(y_true, y_preds):
    accuracy_sum = 0
    precision_sum = 0
    recall_sum = 0
    f1_sum = 0
    y_col_count = 0
    for col in y_true:
        report = classification_report(y_test[col], y_preds[col], output_dict=True)
        print()
        accuracy_sum += report['accuracy']
        precision_sum += report['weighted avg']['precision']
        recall_sum += report['weighted avg']['recall']
        f1_sum += report['weighted avg']['f1-score']
        y_col_count += 1
        
    acc_agg = round(accuracy_sum/y_col_count, 5)
    precision_agg = round(precision_sum/y_col_count, 5)
    recall_agg = round(recall_sum/y_col_count, 5)
    f1_agg = round(f1_sum/y_col_count, 5)
    
    aggregate = {'F1 Accuracy aggregate': acc_agg, 
                 'Precision weighted avg aggregate': precision_agg, 
                 'Recall weighted avg aggregate': recall_agg, 
                 'F1 weighted avg aggregate': f1_agg}
    print('F1 Accuracy aggregate: ', acc_agg)
    print('Precision weighted avg aggregate: ', precision_agg)
    print('Recall weighted avg aggregate: ', recall_agg)
    print('F1 weighted avg aggregate: ', f1_agg)
    return aggregate

In [14]:
default_f1_agg = class_report_values(y_test, default_preds)

NameError: name 'default_preds' is not defined

### 6. Improve your model
Use grid search to find better parameters. 

In [64]:
all_params = dict(pipeline.get_params())
all_params

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7ff6b08e85f0>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                                                        leaf_size=30,
                                                        metric='minkowski',
                                                        metric_params=None,
                                                        n_jobs=None, n_nei

In [67]:
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [17]:
limited_params = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_neighbors': (2, 5, 10)
    }

In [26]:
def best_gridsearch_estimator(parameters):
    para_combinations = 1
    for i in limited_params.values():
        para_combinations = para_combinations*len(i)
    with timer("Gridsearch Fit on {} combinations".format(str(para_combinations))):
        cv = GridSearchCV(pipeline, scoring='neg_mean_squared_error', param_grid=parameters, verbose=10, n_jobs=16)
        grid_result = cv.fit(X_train, y_train)
        return grid_result.best_estimator_

In [28]:
limited_tuning = best_gridsearch_estimator(limited_params)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:  6.1min
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed: 11.8min
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed: 12.1min
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed: 17.9min
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed: 23.5min
[Parallel(n_jobs=16)]: Done  69 out of  90 | elapsed: 29.2min remaining:  8.9min
[Parallel(n_jobs=16)]: Done  79 out of  90 | elapsed: 29.6min remaining:  4.1min
[Parallel(n_jobs=16)]: Done  90 out of  90 | elapsed: 33.0min finished


Gridsearch Fit on 18 combinations executed in 34.03333333333333 minutes



We can train the final model over the entire dataset to improve the final string input UI

In [35]:
def model_pickler(model, model_name):
    with timer('{} model fit and pickle'.format(model_name)):
        model = model.fit(X,y)
        pickle.dump(model, open("{}.pkl".format(model_name), "wb" ) )

In [36]:
model_pickler(limited_tuning, '34m_tuning_model')

34m_tuning model fit and pickle executed in 1.4 minutes



In [39]:
limited_tuning_preds = limited_tuning.predict(X_test)

In [42]:
limited_tuning_df = pd.DataFrame(limited_tuning_preds)
limited_tuning_df.columns = y_columns

In [43]:
class_report_values(y_test, limited_tuning_df)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 Accuracy aggregate:  0.938
Precision weighted avg aggregate:  0.93665
Recall weighted avg aggregate:  0.938
F1 weighted avg aggregate:  0.91795


{'F1 Accuracy aggregate': 0.938,
 'Precision weighted avg aggregate': 0.93665,
 'Recall weighted avg aggregate': 0.938,
 'F1 weighted avg aggregate': 0.91795}

In [47]:
for col in y_columns:
    print("\n****{}****\n".format(col), classification_report(y_test[col], limited_tuning_df[col]))


****related****
               precision    recall  f1-score   support

           0       0.76      0.51      0.61      1532
           1       0.86      0.95      0.90      4986
           2       0.59      0.53      0.56        36

    accuracy                           0.84      6554
   macro avg       0.74      0.66      0.69      6554
weighted avg       0.84      0.84      0.83      6554


****request****
               precision    recall  f1-score   support

           0       0.88      0.99      0.93      5431
           1       0.92      0.32      0.47      1123

    accuracy                           0.88      6554
   macro avg       0.90      0.66      0.70      6554
weighted avg       0.88      0.88      0.85      6554


****offer****
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6554
   macro avg       0.5


****aid_centers****
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.00      0.00      0.00        80

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554


****other_infrastructure****
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6264
           1       0.00      0.00      0.00       290

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.96      0.93      6554


****weather_related****
               precision    recall  f1-score   support

           0       0.77      1.00      0.87      4758
           1       0.96      0.21      0.35      1796

    accuracy                           0.78      6554
   macro avg       0.86      0.60      0.61      

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [314]:
transform_params = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_neighbors': (2, 5, 10)
    }

gen_2_params = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        #'clf__estimator__n_neighbors': (2, 5, 10)
        #'clf__estimator__n_neighbors': (10),
        'clf__estimator__leaf_size': (30, 50),
        'clf__estimator__metric_params': (None, 5),
        'clf__estimator__n_jobs': (None, 5),
        'clf__estimator__n_neighbors': (5, 10)
    }

In [315]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [251]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [48]:
@contextmanager
def timer(process):
    t0 = time.time()
    yield
    if int(time.time() - t0) < 60:
        print('{} executed in {} seconds\n'.format(
            process, int(time.time() - t0)))
    else:
        print('{} executed in {} minutes\n'.format(
            process, int(time.time() - t0) / 60))

In [49]:
def load_data(database_filepath, category_names):
    with timer('Load data'):
        engine = create_engine('sqlite:///{}'.format(database_filepath))
        connection = engine.connect()
        df = Table('labelled_messages', MetaData(),
                autoload=True, autoload_with=engine)
        df = pd.read_sql_table('labelled_messages', connection)

        y = df[category_names]
        X = df['message']
        return X, y, category_names

In [50]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for token in tokens:
        clean_tokens.append(lemmatizer.lemmatize(token).lower().strip())
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words('english')]

    return clean_tokens

In [51]:
def build_model():
    with timer('Build model'):
        pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])
        steps = '    Pipeline steps:'
        for index, step in enumerate(pipeline.steps):
            steps += '\n    Step {}: {}'.format(index+1, step[0])
        print(steps)
        return pipeline

In [60]:
def best_gridsearch_estimator(model, X_train, y_train, parameters):
    with timer("Tune model hyperparameters"):
        para_combinations = 1
        for i in parameters.values():
            para_combinations = para_combinations*len(i)
        with timer("Gridsearch Fit on {} hyperparameter combinations"
                .format(str(para_combinations))):
            cv = GridSearchCV(model, scoring='neg_mean_squared_error',
                            param_grid=parameters, verbose=10, n_jobs=-1)
            grid_result = cv.fit(X_train, y_train)
            return grid_result.best_estimator_

In [61]:
def evaluate_model(model, X_test, y_test, category_names):
    y_preds_array = model.predict(X_test)
    y_preds = pd.DataFrame(y_preds_array)
    y_preds.columns = category_names
    accuracy_sum = 0
    precision_sum = 0
    recall_sum = 0
    f1_sum = 0
    y_col_count = 0
    for col in y_test:
        print("\n****{}****\n".format(col),
              classification_report(y_test[col], y_preds[col]))
        report = classification_report(
            y_test[col], y_preds[col], output_dict=True)
        accuracy_sum += report['accuracy']
        precision_sum += report['weighted avg']['precision']
        recall_sum += report['weighted avg']['recall']
        f1_sum += report['weighted avg']['f1-score']
        y_col_count += 1

    acc_agg = round(accuracy_sum/y_col_count, 5)
    precision_agg = round(precision_sum/y_col_count, 5)
    recall_agg = round(recall_sum/y_col_count, 5)
    f1_agg = round(f1_sum/y_col_count, 5)

    print('----- Average values of entire model -----\n')
    print('F1 Accuracy avg: ', acc_agg)
    print('Precision weighted avg: ', precision_agg)
    print('Recall weighted avg: ', recall_agg)
    print('F1 weighted avg: ', f1_agg)
    print('------------------------------------------\n\n')

In [62]:
def save_model(model, model_filepath, X, y):
    with timer('{} model fit and pickle'.format(model_filepath)):
        model = model.fit(X, y)
        pickle.dump(model, open(model_filepath, "wb"))

In [63]:
def main():
    #if len(sys.argv) == 3:
    database_filepath = '../data/DisasterResponse.db'
    model_filepath = 'tfidf_multioutputclassifer_model.pkl'

    category_names = ['related', 'request', 'offer',
                      'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
                      'security', 'military', 'child_alone', 'water', 'food', 'shelter',
                      'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
                      'infrastructure_related', 'transport', 'buildings', 'electricity',
                      'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
                      'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
                      'other_weather', 'direct_report']
    print('Loading data...\n    DATABASE: {}'.format(database_filepath))
    X, y, category_names = load_data(database_filepath, category_names)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    print('Building model...')
    model = build_model()

    print('Obtaining tuned model hyperparameters...')
    params = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_neighbors': (2, 5, 10)
    }
    best_model = best_gridsearch_estimator(model, X_train, y_train, params)

    print('Training tuned model...')
    with timer("Train with tuned hyperparameters"):
        best_model.fit(X_train, y_train)

    print('Evaluating model...')
    evaluate_model(best_model, X_test, y_test, category_names)

    print('Saving model...\n    MODEL: {}'.format(model_filepath))
    save_model(best_model, model_filepath, X, y)

    print('Trained model saved!')

    #else:
    #    print('Please provide the filepath of the disaster messages database '\
    #          'as the first argument and the filepath of the pickle file to '\
    #          'save the model to as the second argument. \n\nExample: python '\
    #          'train_classifier.py ../data/DisasterResponse.db classifier.pkl')

In [64]:
main()

Loading data...
    DATABASE: ../data/DisasterResponse.db
Load data executed in 0 seconds

Building model...
    Pipeline steps:
    Step 1: vect
    Step 2: tfidf
    Step 3: clf
Build model executed in 0 seconds

Obtaining tuned model hyperparameters...
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


KeyboardInterrupt: 